***Important*** DO NOT CLEAR THE OUTPUT OF THIS NOTEBOOK AFTER EXECUTION!!!

In [ ]:
# if the following command generates an error, you probably didn't enable 
# the cluster security option "Allow API access to all Google Cloud services"
# under Manage Security → Project Access when setting up the cluster
!gcloud dataproc clusters list --region us-central1

# Imports & Setup

In [ ]:
!pip install -q google-cloud-storage==1.43.0
!pip install -q graphframes

In [ ]:
import pyspark
import sys
from collections import Counter, OrderedDict, defaultdict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from pathlib import Path
import pickle
import pandas as pd
from google.cloud import storage
import math
import hashlib
import json

def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

nltk.download('stopwords')

In [ ]:
# if nothing prints here you forgot to include the initialization script when starting the cluster
!ls -l /usr/lib/spark/jars/graph*

In [ ]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf, SparkFiles
from pyspark.sql import SQLContext
from graphframes import *

In [ ]:
spark

In [ ]:
# Put your bucket name below and make sure you can access it without an error
bucket_name = 'ir-project-bucket-313191645-201013310' 
client = storage.Client()
blobs = client.list_blobs(bucket_name)
#for b in blobs:
    #print(b.name)

***GCP setup is complete!*** If you got here without any errors you've earned 10 out of the 35 points of this part.

# Building an inverted index

Here, we read the entire corpus to an rdd, directly from Google Storage Bucket and use your code from Colab to construct an inverted index.

In [ ]:
full_path = "gs://wikidata_preprocessed/*"
parquetFile = spark.read.parquet(full_path)
title_doc_pairs = parquetFile.select("title", "id").rdd# Remark!! when we want to index the title and anchor, add here!!

In [ ]:
# Count number of wiki pages
total_number_of_docs_N = parquetFile.count()

In [ ]:
# if nothing prints here you forgot to upload the file inverted_index_gcp.py to the home dir
%cd -q /home/dataproc
!ls inverted_index_gcp.py

In [ ]:
# adding our python module to the cluster
sc.addFile("/home/dataproc/inverted_index_gcp.py")
sys.path.insert(0,SparkFiles.getRootDirectory())

In [ ]:
from inverted_index_gcp import InvertedIndex
InvertedIndex.DIR_NAME = 'title_index/postings_gcp_title_index'

**YOUR TASK (10 POINTS)**: Use your implementation of `word_count`, `reduce_word_counts`, `calculate_df`, and `partition_postings_and_write` functions from Colab to build an inverted index for all of English Wikipedia in under 2 hours.

A few notes: 
1. The number of corpus stopwords below is a bit bigger than the colab version since we are working on the whole corpus and not just on one file.
2. You need to slightly modify your implementation of  `partition_postings_and_write` because the signature of `InvertedIndex.write_a_posting_list` has changed and now includes an additional argument called `bucket_name` for the target bucket. See the module for more details.
3. You are not allowed to change any of the code not coming from Colab. 

In [ ]:
english_stopwords = frozenset(stopwords.words('english'))
corpus_stopwords = ["category", "references", "also", "external", "links", 
                    "may", "first", "see", "history", "people", "one", "two", 
                    "part", "thumb", "including", "second", "following", 
                    "many", "however", "would", "became"]
FILTERED = False
POSTING_LEN_THRESH = 300
TFIDF_ROUND_THRESH = 4
TFIDF_MUL = 10**TFIDF_ROUND_THRESH # 10000
DIR_NAME = "title_index/postings_gcp_title_index"

all_stopwords = english_stopwords.union(corpus_stopwords)
RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)

from functools import total_ordering

NUM_BUCKETS = 124
def token2bucket_id(token):
  return int(_hash(token),16) % NUM_BUCKETS

def word_count(text, id):
  tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
  if FILTERED:
      set_of_toks = set(tokens)
      len_before = len(set_of_toks)
      len_after = len(set_of_toks - test_set_terms)
      if len_after == len_before:
        return []
  tf_counter = Counter()
  for tok in tokens:
    if tok not in all_stopwords:
      tf_counter[tok] += 1
  to_ret = []
  for item in tf_counter.items():
    to_ret.append((item[0],(id,item[1])))
  return to_ret

def reduce_word_counts(unsorted_pl):
  return sorted(unsorted_pl,key=lambda x: x[0])

#start addition-------------------------
def get_posting_size(n):
    if POSTING_LEN_THRESH > n:
        return n
    return POSTING_LEN_THRESH

def our_sum(lst):
  summ = 0
  for i in lst:
    summ+=i
  return summ
#end addition------------------------------

def partition_postings_and_write(postings):
  temp = postings.map(lambda x: (token2bucket_id(x[0]),(x[0],x[1])))
  temp = temp.groupByKey()
  return temp.map(lambda x: InvertedIndex.write_a_posting_list(x,bucket_name))


In [ ]:
#start addition-------------------------
doc_tf_vector = title_doc_pairs.map(lambda x: (x[1], word_count(x[0], x[1]))).filter(lambda x: len(x[1]) > 0)
doc_tf_vector_with_doc_len = doc_tf_vector.map(lambda x: (x[0], our_sum([i[1][1] for i in x[1]]),x[1]))
doc_tf_div_by_doc_len = doc_tf_vector_with_doc_len.map(lambda x: (x[0],[(term,(tup[0],tup[1]/x[1])) for term, tup in x[2]]))

flating = doc_tf_div_by_doc_len.map(lambda x: x[1]).flatMap(lambda x: x).groupByKey().mapValues(list)

add_df_for_value = flating.mapValues(lambda x: (len(x),x)).filter(lambda x: x[1][0] > 10)

posting_list_befor_trans = add_df_for_value.mapValues(lambda x: [(doc_id, tf*math.log(total_number_of_docs_N/x[0],2)) for doc_id, tf in x[1]])

doc_to_vec =  posting_list_befor_trans.map(lambda x: x[1]).flatMap(lambda x: x).groupByKey().mapValues(list)
doc_vec_size_rdd = doc_to_vec.mapValues(lambda x: our_sum([i**2 for i in x])**0.5)
#end addition------------------------------

In [ ]:
doc_vec_size_dict  = dict(doc_vec_size_rdd.collect())

In [ ]:
posting_list_div_by_doc_vec_size = posting_list_befor_trans.mapValues(lambda x: [(doc_id, tfidf/doc_vec_size_dict[doc_id]) for doc_id, tfidf in x])

In [ ]:
rounded_posting_lists = posting_list_div_by_doc_vec_size.mapValues(lambda x: [(doc_id, int(tf_idf*10000)) for doc_id, tf_idf in x])

sorted_posting_list = rounded_posting_lists.mapValues(lambda x: sorted(x, key=lambda y: y[1], reverse=True))

# slicing posting list to POSTING_LEN_THRESH size (if posting list len is bigger)
final_posting_lists = sorted_posting_list.map(lambda x: (x[0], x[1][:get_posting_size(len(x[1]))]))
print('finish')

In [ ]:
_ = partition_postings_and_write(final_posting_lists).collect()

In [ ]:
super_posting_locs = defaultdict(list)
for blob in client.list_blobs(bucket_name, prefix=DIR_NAME):
  if not blob.name.endswith("pickle"):
    continue
  print("found " + blob.name)
  with blob.open("rb") as f:
    posting_locs = pickle.load(f)
    for k, v in posting_locs.items():
      super_posting_locs[k].extend(v)
    print("wrote " + blob.name)
print("finish2")

In [ ]:
df_dict_rdd = final_posting_lists.mapValues(lambda x: len(x))
df_dict = dict(df_dict_rdd.collect())
print(df_dict["door"])
print(len(df_dict))

In [ ]:
title_index = InvertedIndex()
title_index.df = df_dict
title_index.posting_locs = super_posting_locs

In [ ]:
# write the global stats out
title_index.write_index('.', 'title_index')
# upload to gs
index_src = "title_index.pkl"
index_dst = f'gs://{bucket_name}/title_index/postings_tfidf_gcp/{index_src}'
!gsutil cp $index_src $index_dst

In [ ]:
!gsutil ls -lh $index_dst

In [ ]:
title_index.posting_locs['silicon']